![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/zero-shot%20text%20classification/DeBertaForZeroShotClassification.ipynb)

## Colab Setup

In [1]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.1
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.9 MB/s eta 0:00:00


# Download MPNetForQuestionAnswering Model and Create Spark NLP Pipeline

Lets create a Spark NLP pipeline with the following stages:

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

# for GPU training >> sparknlp.start(gpu = True)
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 5.3.1
Apache Spark version: 3.2.3


In [10]:
DeBertaForZeroShotClassification

sparknlp.annotator.classifier_dl.deberta_for_zero_shot_classification.DeBertaForZeroShotClassification

In [36]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

zeroShotClassifier = DeBertaForZeroShotClassification \
    .pretrained('deberta_base_zero_shot_classifier_mnli_anli_v3', 'en') \
    .setInputCols(['token', 'document']) \
    .setOutputCol('class') \
    .setCaseSensitive(False) \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])


pipeline = Pipeline(stages=[
        document_assembler,
        tokenizer,
        zeroShotClassifier
])


deberta_base_zero_shot_classifier_mnli_anli_v3 download started this may take some time.
Approximate size to download 420.7 MB
[OK!]


Lets create a dataframe with some queries to be used as input for the pipeline.

In [37]:
examples = [
    ['I urgently need advice on handling a sudden job offer decision!'],
    ["My phone screen just went black and won't turn on, I need a fix asap!"],
    ["I'm about to miss my connecting flight due to a delay, what are my options?"],
    ["I'm hosting movie night tomorrow and my projector stopped working, help!"],
    ['My playlist got deleted right before the party, how can I recover it quickly?'],
    ['The championship game is tonight and my TV signal is out, what can I do?'],
    ["A storm is coming and I'm not prepared, how can I quickly secure my home?"],
    ['My computer got a virus right before my project deadline, I need immediate assistance!']
]

data = spark.createDataFrame(examples).toDF("text")

display the results

In [38]:
result = pipeline.fit(data).transform(data)
result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------+--------+
|text                                                                                  |result  |
+--------------------------------------------------------------------------------------+--------+
|I urgently need advice on handling a sudden job offer decision!                       |[mobile]|
|My phone screen just went black and won't turn on, I need a fix asap!                 |[travel]|
|I'm about to miss my connecting flight due to a delay, what are my options?           |[sport] |
|I'm hosting movie night tomorrow and my projector stopped working, help!              |[movie] |
|My playlist got deleted right before the party, how can I recover it quickly?         |[mobile]|
|The championship game is tonight and my TV signal is out, what can I do?              |[sport] |
|A storm is coming and I'm not prepared, how can I quickly secure my home?             |[travel]|
|My computer got a v